In [ ]:
#Debugging 13-10-2023 == FALSE_V1; LATEST STABLE == V1
#UI == pygame
#Implementado gráfico Max-Boltz
#'future' funciona (inherited 6_10)
#Colisão inelástica e elástica implementada
#Colisão futura implementada em classe Particul
#Reação química feita + grafico conc./tempo
#Fazer: encontrar a lei de velocidade (derivadas)
#Problemas: 
# -não usa grid (menos eficiente) -> James: sistema para armazenar vizinhos
# -NA_CL_INDEX como global
# -a = 'Na', b = 'Cl'; a = 'Cl', b = 'Na'-> e se forem 3?
# -intersection_position_checker que nao precisa checar tudo ?
# -reações que geram partículas grandes podem gerar intersecção com outra que está indo em direção a esta


import math
import random
import warnings
import pygame
from pygame.locals import *
from sys import exit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Constants
FPS = 60
DT = 1/FPS
ALTURA_CAIXA = 500
LARGURA_CAIXA = 500
NUM_PARTICULAS = 30
GRID_SIZE = 10
FUTURE_FACTOR = 2
TIPO_COLISAO = 'elastic'
COEFICIENTE_RESTITUICAO = 0.0 #coeficiente de restituição

NAME_BOLTZ = 'max-boltz_6_15.png'
NAME_CONCENTRATION_PLOT = 'concentration_6_15.png'
NAME_CSV = 'concentration_data_6_15.csv'

# Indexes de moléculas das reações
NA_CL_INDEX = 0
CHANCE_NACL_GLOBAL = 0.4 #chances de não acontecer a reação

# Colors
WHITE = (255, 255, 255)

class Particula:
    def __init__(self, p_type, massa, raio, vel_x, vel_y, x, y, cor):
        self.p_type = p_type
        self.massa = float(massa)
        self.raio = float(raio)
        self.vel_x = float(vel_x)
        self.vel_y = float(vel_y)
        self.x = float(x)
        self.y = float(y)
        self.x_future = None
        self.y_future = None
        self.cor = cor
        self.sprite_pygame = self.sprite_pygame()

    def deslocar(self):

        self.x += self.vel_x * DT
        self.y += self.vel_y * DT

    def next_position(self):
        self.x_future = self.x + self.vel_x * DT * FUTURE_FACTOR
        self.y_future = self.y + self.vel_y * DT * FUTURE_FACTOR

        if self.x_future < self.raio or self.x_future > LARGURA_CAIXA - self.raio:
            self.vel_x *= -1
        if self.y_future < self.raio or self.y_future > ALTURA_CAIXA - self.raio:
            self.vel_y *= -1

        #print('next_position')

    def sprite_pygame(self):
        circle = pygame.Surface((self.raio * 2, self.raio * 2), pygame.SRCALPHA)
        pygame.draw.circle(circle, self.cor, (self.raio, self.raio), self.raio)
        sprite = pygame.sprite.Sprite()
        sprite.image = circle
        sprite.rect = sprite.image.get_rect(center=(self.x, self.y))
        return sprite


class Grid:
    def __init__(self, width, height, cell_size):
        self.width = width
        self.height = height
        self.cell_size = cell_size
        self.num_cols = int(width / cell_size)
        self.num_rows = int(height / cell_size)
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #cria uma grid nova

    def clear(self):
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #'limpa' a grid

    def index(self, x, y):
        col = int(x / self.cell_size)
        row = int(y / self.cell_size)
        
        # Ensure col and row are within bounds
        col = max(0, min(col, self.num_cols - 1))
        row = max(0, min(row, self.num_rows - 1))
        
        return col + row * self.num_cols

    def insert(self, particle):
        index = self.index(particle.x, particle.y)
        self.grid[index].append(particle)

    def get_particles_in_cell(self, x, y):
        index = self.index(x, y)
        return self.grid[index]

    def update_grid(self, particles):
        self.clear()
        for particle in particles.values():
            self.insert(particle)

    def check_grid_collision(self,particles, grid, sprites):
        particle_dict = particles
        collision_occurred = False  # Add a flag to check if any collision occurred

        for name_p1, particle_p1 in particles.items():
            cell_x = int(particle_p1.x / grid.cell_size)
            cell_y = int(particle_p1.y / grid.cell_size)

            for dx in [-1, 0, 1]:
                for dy in [-1, 0, 1]:
                    neighbor_x = cell_x + dx
                    neighbor_y = cell_y + dy
                    if 0 <= neighbor_x < grid.num_cols and 0 <= neighbor_y < grid.num_rows:
                        for name_p2, particle_p2 in particles.items():
                            if particle_p1 != particle_p2:
                                if check_collision(particle_p1, particle_p2):
                                    particles = reaction_collision(particle_dict, particle_p1, name_p1, particle_p2, name_p2, sprites)
                                    collision_occurred = True

        if not collision_occurred:  # If no collision occurred, return the unchanged particles
            return particles

        return particles  # Return the modified particles if a collision occurred



def check_collision(p1, p2):

    p1.next_position() #proxima_posicao

    p2.next_position() #proxima_posicao

    distance_future = math.sqrt((p2.x_future - p1.x_future)**2 + (p2.y_future - p1.y_future)**2)
    #distance = math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
    if (distance_future <= p1.raio + p2.raio) == True:
        return True
    else:
        return False


def reaction_collision(particles, p1, p1_index, p2, p2_index, sprites):

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    vel_new_particle = ((p1.massa * v1) + (p2.massa * v2)) / (p1.massa + p2.massa) #apenas momento conservado

    #print('arrays_reaction_v:', v1, v2)

    CM_x = (p1.massa * p1.x + p2.massa * p2.x) / (p1.massa + p2.massa)
    CM_y = (p1.massa * p1.y + p2.massa * p2.y) / (p1.massa + p2.massa)

    chance_NaCl_local = random.random() #chanc de acontecer a reação

    #print('arrays_reaction_CM:', CM_x, CM_y )

    #new_particles = particles.copy()
    #print('old_particles:', particles)


    if ((p1.p_type == 'Na' and p2.p_type == 'Cl') or (p1.p_type == 'Cl' and p2.p_type == 'Na')) and (chance_NaCl_local > CHANCE_NACL_GLOBAL):
        #print(p1.p_type, p2.p_type, 'form NaCl')

        global NA_CL_INDEX

        p_type = 'NaCl'
        massa = (p1.massa + p2.massa)
        raio = 5
        vel_x = vel_new_particle[0]
        vel_y = vel_new_particle[1]
        x = CM_x
        y = CM_y
        #x_future = x + vel_x * DT * FUTURE_FACTOR
        #y_future = y + vel_y * DT * FUTURE_FACTOR
        cor = (125, 125, 125)
        nome_particula = f"{p_type}_{NA_CL_INDEX}" 
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)

        sprites.add(particle_instance.sprite_pygame)
        
        NA_CL_INDEX += 1

        particles.update({nome_particula : particle_instance})
        particles.pop(p1_index)
        particles.pop(p2_index)

        p1.sprite_pygame.kill()
        p2.sprite_pygame.kill()


    else:
        #print('does not react')
        resolve_collision(p1,p2,collision_type=TIPO_COLISAO)

    #print('new_particles:',particles)
    #print()
    return particles


def resolve_collision(p1, p2, collision_type='elastic'):
    x1 = np.array([p1.x, p1.y])
    x2 = np.array([p2.x, p2.y])

    #print('arrays_x:', x1, x2)

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    #print('arrays_v:', v1, v2)

    if collision_type == 'elastic' or collision_type == 'elastica':
        #print('elastica')

        C = 1

    if collision_type == 'partial_inelastic' or collision_type == 'parcial_inelastica':
        #print('inelastica')

        C = COEFICIENTE_RESTITUICAO

    new_v1 = v1 - ((((C * p2.massa) + p2.massa) / (p1.massa + p2.massa)) * ((np.dot((v1 - v2),(x1 - x2))) / ((np.linalg.norm(x1 - x2))**2)) * (x1 - x2))
    new_v2 = v2 - ((((C * p1.massa) + p1.massa) / (p1.massa + p2.massa)) * ((np.dot((v2 - v1),(x2 - x1))) / ((np.linalg.norm(x2 - x1))**2)) * (x2 - x1))

    p1.vel_x = new_v1[0]
    p1.vel_y = new_v1[1]
    p2.vel_x = new_v2[0]
    p2.vel_y = new_v2[1]

    #print('resolution',p1.vel_x, p1.vel_y, p2.vel_x, p2.vel_y)

#src_elastica = https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
#src_inelastica = https://physics.stackexchange.com/questions/708495/angle-free-two-dimensional-inelastic-collision-formula




def gerar_particula(n_particulas):
    particulas = {}

    #atomos
    atomos = {
        "Na": {"massa": 22, "raio": 5, "color": (255, 0, 0)}, #approximations
        "Cl": {"massa": 35, "raio": 5, "color": (0, 255, 0)},

    }

    for particle_index in range(1, n_particulas + 1):
        #escolha de distribuição aleatória
        atomos_selecionar = random.choice(list(atomos.keys()))

        attributes = atomos[atomos_selecionar]

        p_type = atomos_selecionar
        massa = attributes["massa"]
        raio = attributes["raio"]
        vel_x = random.uniform(-60, 60) #velocidade inicial não especificada
        vel_y = random.uniform(-60, 60)
        x = random.uniform(raio, LARGURA_CAIXA - raio)
        y = random.uniform(raio, ALTURA_CAIXA - raio)
        #x_future = x + vel_x * DT * FUTURE_FACTOR
        #y_future = y + vel_y * DT * FUTURE_FACTOR
        cor = attributes["color"]
        nome_particula = f"{atomos_selecionar}_{particle_index}" 
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)
        particulas[nome_particula] = particle_instance

    return particulas

def select_particles(particles,sprites):
    particles_update = set()
    particles_remove = []

    #print('entrou pra selected')

    particle_pairs = np.array(list(itertools.combinations(particles.items(), 2)))
    #print('pares',particle_pairs)

    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        #print((name_p1, type(instance_p1)), (name_p2, type(instance_p2)))
        if check_collision(instance_p1, instance_p2):
            #print('foi pra reacted')
            reacted_particles = reaction_collision(particles, instance_p1, name_p1, instance_p2, name_p2,sprites)
            

            if particles != reacted_particles:
                particles_update.update(reacted_particles)
                particles_remove.extend([name_p1, name_p2])

    for particle_name in particles_remove:
        del particles[particle_name]

    particles.update(particles_update)

    return particles



def intersection_pos_checker(particles,iteration_max = 100):
    iteration_count = 0
    intersection = False
    particle_pairs = np.array(list(itertools.permutations(particles.items(), 2)))
    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        if check_collision(instance_p1, instance_p2):
                particle_move = random.choice([[name_p1,instance_p1], [name_p2,instance_p2]])
                new_x = random.uniform(particle_move[1].raio, LARGURA_CAIXA - (particle_move[1].raio))
                new_y = random.uniform(particle_move[1].raio, ALTURA_CAIXA - (particle_move[1].raio))
                particles[particle_move[0]].x = new_x
                particles[particle_move[0]].y = new_y
                iteration_count +=1
                #print(particles[particle_move[0]].y) #verificar coordenadas alteradas

                intersection = True
                if iteration_max == iteration_count:
                    warnings.warn(f'{iteration_max} iterações atingidas. Deve haver partículas sobrepostas. Altere o tamanho da caixa.')
                    return particles
    
    if intersection == True:
        #print('True')
        return intersection_pos_checker(particles)

    if intersection == False:
        #print('False')
        return particles

def create_particle_group_sprites(particles):
    sprites = pygame.sprite.Group()
    for particle_instance in particles.values():
        sprite = particle_instance.sprite_pygame
        sprites.add(sprite)
    #print(sprites)
    return sprites


def chemical_counter(particles,time_passed,df=pd.DataFrame()):

    chem_list = particles.keys()
    #n_moleculas = len(chem_list)

    element_count = {}  # Initialize an empty dictionary to store element counts

    for item in chem_list:
        element = item.split('_')[0]  # Extract the element (part before the underscore)
        if element in element_count:
            element_count[element] += 1  # Increment the count if the element already exists
        else:
            element_count[element] = 1  # Initialize count to 1 if the element is encountered for the first time

   # for key in element_count: #transforma em %
    #    element_count[key] /= n_moleculas


    #df_chem_columns = np.append('Tempo',list(element_count.keys()))
    #print(df_chem_columns)
    chem_array = np.append(time_passed,list(element_count.values()))
    #print(chem_array)
    df_chem = pd.DataFrame(chem_array).T
    df_chem.columns = np.append('Tempo',list(element_count.keys()))

    df_chem = pd.concat([df, df_chem], axis=0, ignore_index=True)

    return df_chem
    

def maxwell_boltzmann_plot(final_speeds,name):
    # Barplot/histograma das velocidades Max-Boltz.
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    plt.hist(final_speeds, bins=25, color='blue', edgecolor='black', alpha=0.7)
    plt.xlabel('Velocidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das velocidades finais')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def concentration_plot(df_chem,name):
    # Lineplot das concentrações/tempo.
    df_chem.to_csv(NAME_CSV, index=False)
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    for y_axis, labels in zip(df_chem.columns[1:],list(df_chem.columns)[1:]):
        plt.plot(df_chem['Tempo'],df_chem[y_axis] ,alpha=0.7, label=labels)
    plt.legend()
    plt.xlabel('Tempo [s]')
    plt.ylabel('Quantidade [un]')
    plt.title('Evolução temporal das concentrações')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def main():
    final_speeds = []
    pygame.init()
    screen = pygame.display.set_mode((LARGURA_CAIXA, ALTURA_CAIXA))
    pygame.display.set_caption("Collision Simulation")

    particles = gerar_particula(NUM_PARTICULAS)
    particles = intersection_pos_checker(particles)
    sprites = create_particle_group_sprites(particles)

    #digito_atual = 0
    time_passed = 0
    df_chem = chemical_counter(particles, time_passed)

    #grid = Grid(LARGURA_CAIXA, ALTURA_CAIXA, GRID_SIZE)

    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                # Pegar velocidades finais de todas as particulas
                for particle_instance in particles.values():
                    final_speed = math.sqrt(particle_instance.vel_x**2 + particle_instance.vel_y**2)
                    final_speeds.append(final_speed)

                maxwell_boltzmann_plot(final_speeds,name=NAME_BOLTZ) #desligado para debugging

                df_chem = df_chem.fillna(0)
                concentration_plot(df_chem,name=NAME_CONCENTRATION_PLOT)
                exit()

        time_passed += DT
        #primeiro_digito = int(time_passed)

        #if (digito_atual == 0) or (digito_atual != primeiro_digito):
         #   primeiro_digito = digito_atual
            #print('novo df_chem')
        df_chem = chemical_counter(particles,time_passed,df_chem)
            

        #print('after_reaction')
        particles = select_particles(particles,sprites)
        #print(type(after_reaction_particles))

        # Update particle positions after collision detection
        for particle_name, particle_instance in particles.items():
            particle_instance.deslocar()
            particle_instance.next_position() #usado para calcular if colisao == True
            #print('Debugger:',particle_instance.x, particle_instance.y, particle_instance.x_future, particle_instance.y_future)

        # Update sprite positions
        for sprite, particle_instance in zip(sprites, particles.values()):
            sprite.rect.center = (particle_instance.x, particle_instance.y)

        #print(time_passed)

        screen.fill(WHITE)
        sprites.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

if __name__ == "__main__":
    main()


In [10]:
#Debugging 13-10-2023 == FALSE_V1; LATEST STABLE == V1
#UI == pygame
#Implementado gráfico Max-Boltz
#'future' funciona (inherited 6_10)
#Colisão inelástica e elástica implementada
#Colisão futura implementada em classe Particul
#Reação química feita + grafico conc./tempo
#Fazer: encontrar a lei de velocidade (derivadas)
#Problemas: 
# -não usa grid (menos eficiente)
# -NA_CL_INDEX como global
# -a = 'Na', b = 'Cl'; a = 'Cl', b = 'Na'-> e se forem 3?
# -intersection_position_checker que nao precisa checar tudo ?
# -reações que geram partículas grandes podem gerar intersecção com outra que está indo em direção a esta


import math
import random
import warnings
import pygame
from pygame.locals import *
from sys import exit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Constants
FPS = 60
DT = 1/FPS
ALTURA_CAIXA = 500
LARGURA_CAIXA = 500
NUM_PARTICULAS = 30
GRID_SIZE = 10
FUTURE_FACTOR = 2
TIPO_COLISAO = 'elastic'
COEFICIENTE_RESTITUICAO = 0.0 #coeficiente de restituição

NAME_BOLTZ = 'max-boltz_6_15.png'
NAME_CONCENTRATION_PLOT = 'concentration_6_15.png'
NAME_CSV = 'concentration_data_6_15.csv'

# Indexes de moléculas das reações
NA_CL_INDEX = 0
CHANCE_NACL_GLOBAL = 0.4 #chances de não acontecer a reação

# Colors
WHITE = (255, 255, 255)

class Particula:
    def __init__(self, p_type, massa, raio, vel_x, vel_y, x, y, cor):
        self.p_type = p_type
        self.massa = float(massa)
        self.raio = float(raio)
        self.vel_x = float(vel_x)
        self.vel_y = float(vel_y)
        self.x = float(x)
        self.y = float(y)
        self.x_future = None
        self.y_future = None
        self.cor = cor
        self.sprite_pygame = self.sprite_pygame()

    def deslocar(self):

        self.x += self.vel_x * DT
        self.y += self.vel_y * DT

    def next_position(self):
        self.x_future = self.x + self.vel_x * DT * FUTURE_FACTOR
        self.y_future = self.y + self.vel_y * DT * FUTURE_FACTOR

        if self.x_future < self.raio or self.x_future > LARGURA_CAIXA - self.raio:
            self.vel_x *= -1
        if self.y_future < self.raio or self.y_future > ALTURA_CAIXA - self.raio:
            self.vel_y *= -1

        #print('next_position')

    def sprite_pygame(self):
        circle = pygame.Surface((self.raio * 2, self.raio * 2), pygame.SRCALPHA)
        pygame.draw.circle(circle, self.cor, (self.raio, self.raio), self.raio)
        sprite = pygame.sprite.Sprite()
        sprite.image = circle
        sprite.rect = sprite.image.get_rect(center=(self.x, self.y))
        return sprite


class Grid:
    def __init__(self, width, height, cell_size):
        self.width = width
        self.height = height
        self.cell_size = cell_size
        self.num_cols = int(width / cell_size)
        self.num_rows = int(height / cell_size)
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #cria uma grid nova

    def clear(self):
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #'limpa' a grid

    def index(self, x, y):
        col = int(x / self.cell_size)
        row = int(y / self.cell_size)
        
        # Ensure col and row are within bounds
        col = max(0, min(col, self.num_cols - 1))
        row = max(0, min(row, self.num_rows - 1))
        
        return col + row * self.num_cols

    def insert(self, particle):
        index = self.index(particle.x, particle.y)
        self.grid[index].append(particle)

    def get_particles_in_cell(self, x, y):
        index = self.index(x, y)
        return self.grid[index]

# The updated check_grid_collision function
    def check_grid_collision(self,particles, grid, sprites):
        particles_to_remove = []  # Create a list to store keys to be removed
        particles_to_add = {}  # Create a dictionary to store items to be added

        for name_p1, particle_p1 in particles.items():
            cell_x = int(particle_p1.x / grid.cell_size)
            cell_y = int(particle_p1.y / grid.cell_size)

            for dx, dy in itertools.product([-1, 0, 1], repeat=2):
                neighbor_x = cell_x + dx
                neighbor_y = cell_y + dy
                if 0 <= neighbor_x < grid.num_cols and 0 <= neighbor_y < grid.num_rows:
                    for name_p2, particle_p2 in particles.items():
                        if particle_p1 != particle_p2:
                            if check_collision(particle_p1, particle_p2):
                                new_particles, particles_removed = reaction_collision(
                                    particles, particle_p1, name_p1, particle_p2, name_p2, sprites
                            )
                            # Store keys to be removed
                                particles_to_remove.extend(particles_removed)

                            # Store items to be added
                                particles_to_add.update(new_particles)

    # Apply the changes after the for loops
        for key in particles_to_remove:
            del particles[key]

        particles.update(particles_to_add)

        return particles




def check_collision(p1, p2):

    p1.next_position() #proxima_posicao

    p2.next_position() #proxima_posicao

    distance_future = math.sqrt((p2.x_future - p1.x_future)**2 + (p2.y_future - p1.y_future)**2)
    #distance = math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
    if (distance_future <= p1.raio + p2.raio) == True:
        return True
    else:
        return False


def reaction_collision(particles, p1, p1_name, p2, p2_name, sprites):

    global NA_CL_INDEX

    particles_to_add = {}  # Create a dictionary to store new particles
    particles_to_remove = []  # Create a list to store particles to be removed

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    vel_new_particle = ((p1.massa * v1) + (p2.massa * v2)) / (p1.massa + p2.massa)

    CM_x = (p1.massa * p1.x + p2.massa * p2.x) / (p1.massa + p2.massa)
    CM_y = (p1.massa * p1.y + p2.massa * p2.y) / (p1.massa + p2.massa)

    chance_NaCl_local = random.random()

    if ((p1.p_type == 'Na' and p2.p_type == 'Cl') or (p1.p_type == 'Cl' and p2.p_type == 'Na')) and (chance_NaCl_local > CHANCE_NACL_GLOBAL):
        p_type = 'NaCl'
        massa = (p1.massa + p2.massa)
        raio = 5
        vel_x = vel_new_particle[0]
        vel_y = vel_new_particle[1]
        x = CM_x
        y = CM_y
        cor = (125, 125, 125)
        name_particle = f"{p_type}_{NA_CL_INDEX}"
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)

        sprites.add(particle_instance.sprite_pygame)

        p1.sprite_pygame.kill()
        p2.sprite_pygame.kill()

        particles_to_add[name_particle] = particle_instance
        particles_to_remove.extend([p1_name, p2_name])
        NA_CL_INDEX += 1
    else:
        resolve_collision(p1, p2, collision_type=TIPO_COLISAO)

    return particles_to_add, particles_to_remove



def resolve_collision(p1, p2, collision_type='elastic'):
    x1 = np.array([p1.x, p1.y])
    x2 = np.array([p2.x, p2.y])

    #print('arrays_x:', x1, x2)

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    #print('arrays_v:', v1, v2)

    if collision_type == 'elastic' or collision_type == 'elastica':
        #print('elastica')

        C = 1

    if collision_type == 'partial_inelastic' or collision_type == 'parcial_inelastica':
        #print('inelastica')

        C = COEFICIENTE_RESTITUICAO

    new_v1 = v1 - ((((C * p2.massa) + p2.massa) / (p1.massa + p2.massa)) * ((np.dot((v1 - v2),(x1 - x2))) / ((np.linalg.norm(x1 - x2))**2)) * (x1 - x2))
    new_v2 = v2 - ((((C * p1.massa) + p1.massa) / (p1.massa + p2.massa)) * ((np.dot((v2 - v1),(x2 - x1))) / ((np.linalg.norm(x2 - x1))**2)) * (x2 - x1))

    p1.vel_x = new_v1[0]
    p1.vel_y = new_v1[1]
    p2.vel_x = new_v2[0]
    p2.vel_y = new_v2[1]

    #print('resolution',p1.vel_x, p1.vel_y, p2.vel_x, p2.vel_y)

#src_elastica = https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
#src_inelastica = https://physics.stackexchange.com/questions/708495/angle-free-two-dimensional-inelastic-collision-formula




def gerar_particula(n_particulas):
    particulas = {}

    #atomos
    atomos = {
        "Na": {"massa": 22, "raio": 5, "color": (255, 0, 0)}, #approximations
        "Cl": {"massa": 35, "raio": 5, "color": (0, 255, 0)},

    }

    for particle_index in range(1, n_particulas + 1):
        #escolha de distribuição aleatória
        atomos_selecionar = random.choice(list(atomos.keys()))

        attributes = atomos[atomos_selecionar]

        p_type = atomos_selecionar
        massa = attributes["massa"]
        raio = attributes["raio"]
        vel_x = random.uniform(-60, 60) #velocidade inicial não especificada
        vel_y = random.uniform(-60, 60)
        x = random.uniform(raio, LARGURA_CAIXA - raio)
        y = random.uniform(raio, ALTURA_CAIXA - raio)
        #x_future = x + vel_x * DT * FUTURE_FACTOR
        #y_future = y + vel_y * DT * FUTURE_FACTOR
        cor = attributes["color"]
        nome_particula = f"{atomos_selecionar}_{particle_index}" 
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)
        particulas[nome_particula] = particle_instance

    return particulas

def select_particles(particles,sprites):
    particles_update = set()
    particles_remove = []

    #print('entrou pra selected')

    particle_pairs = np.array(list(itertools.combinations(particles.items(), 2)))
    #print('pares',particle_pairs)

    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        #print((name_p1, type(instance_p1)), (name_p2, type(instance_p2)))
        if check_collision(instance_p1, instance_p2):
            #print('foi pra reacted')
            reacted_particles = reaction_collision(particles, instance_p1, name_p1, instance_p2, name_p2,sprites)
            

            if particles != reacted_particles:
                particles_update.update(reacted_particles)
                particles_remove.extend([name_p1, name_p2])

    for particle_name in particles_remove:
        del particles[particle_name]

    particles.update(particles_update)

    return particles



def intersection_pos_checker(particles,iteration_max = 100):
    iteration_count = 0
    intersection = False
    particle_pairs = np.array(list(itertools.permutations(particles.items(), 2)))
    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        if check_collision(instance_p1, instance_p2):
                particle_move = random.choice([[name_p1,instance_p1], [name_p2,instance_p2]])
                new_x = random.uniform(particle_move[1].raio, LARGURA_CAIXA - (particle_move[1].raio))
                new_y = random.uniform(particle_move[1].raio, ALTURA_CAIXA - (particle_move[1].raio))
                particles[particle_move[0]].x = new_x
                particles[particle_move[0]].y = new_y
                iteration_count +=1
                #print(particles[particle_move[0]].y) #verificar coordenadas alteradas

                intersection = True
                if iteration_max == iteration_count:
                    warnings.warn(f'{iteration_max} iterações atingidas. Deve haver partículas sobrepostas. Altere o tamanho da caixa.')
                    return particles
    
    if intersection == True:
        #print('True')
        return intersection_pos_checker(particles)

    if intersection == False:
        #print('False')
        return particles

def create_particle_group_sprites(particles):
    sprites = pygame.sprite.Group()
    for particle_instance in particles.values():
        sprite = particle_instance.sprite_pygame
        sprites.add(sprite)
    #print(sprites)
    return sprites


def chemical_counter(particles,time_passed,df=pd.DataFrame()):

    chem_list = particles.keys()
    #n_moleculas = len(chem_list)

    element_count = {}  # Initialize an empty dictionary to store element counts

    for item in chem_list:
        element = item.split('_')[0]  # Extract the element (part before the underscore)
        if element in element_count:
            element_count[element] += 1  # Increment the count if the element already exists
        else:
            element_count[element] = 1  # Initialize count to 1 if the element is encountered for the first time

   # for key in element_count: #transforma em %
    #    element_count[key] /= n_moleculas


    #df_chem_columns = np.append('Tempo',list(element_count.keys()))
    #print(df_chem_columns)
    chem_array = np.append(time_passed,list(element_count.values()))
    #print(chem_array)
    df_chem = pd.DataFrame(chem_array).T
    df_chem.columns = np.append('Tempo',list(element_count.keys()))

    df_chem = pd.concat([df, df_chem], axis=0, ignore_index=True)

    return df_chem
    

def maxwell_boltzmann_plot(final_speeds,name):
    # Barplot/histograma das velocidades Max-Boltz.
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    plt.hist(final_speeds, bins=25, color='blue', edgecolor='black', alpha=0.7)
    plt.xlabel('Velocidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das velocidades finais')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def concentration_plot(df_chem,name):
    # Lineplot das concentrações/tempo.
    df_chem.to_csv(NAME_CSV, index=False)
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    for y_axis, labels in zip(df_chem.columns[1:],list(df_chem.columns)[1:]):
        plt.plot(df_chem['Tempo'],df_chem[y_axis] ,alpha=0.7, label=labels)
    plt.legend()
    plt.xlabel('Tempo [s]')
    plt.ylabel('Quantidade [un]')
    plt.title('Evolução temporal das concentrações')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def main():
    final_speeds = []
    pygame.init()
    screen = pygame.display.set_mode((LARGURA_CAIXA, ALTURA_CAIXA))
    pygame.display.set_caption("Collision Simulation")

    particles = gerar_particula(NUM_PARTICULAS)
    particles = intersection_pos_checker(particles)
    sprites = create_particle_group_sprites(particles)
    grid = Grid(LARGURA_CAIXA, ALTURA_CAIXA, GRID_SIZE)  # Initialize the grid

    time_passed = 0
    df_chem = pd.DataFrame(columns=['Tempo'])  # Create an empty DataFrame for chemical data

    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                # Get final speeds of all particles
                for particle_instance in particles.values():
                    final_speed = math.sqrt(particle_instance.vel_x ** 2 + particle_instance.vel_y ** 2)
                    final_speeds.append(final_speed)

                maxwell_boltzmann_plot(final_speeds, name=NAME_BOLTZ)  # Turned off for debugging

                df_chem = df_chem.fillna(0)
                concentration_plot(df_chem, name=NAME_CONCENTRATION_PLOT)
                exit()

        time_passed += DT

        df_chem = chemical_counter(particles, time_passed, df_chem)
        grid.clear()  # Clear the grid

        for particle_instance in particles.values():
            grid.insert(particle_instance)

        # Use the updated check_grid_collision function
        particles = grid.check_grid_collision(particles, grid, sprites)

        for particle_name, particle_instance in particles.items():
            particle_instance.deslocar()
            particle_instance.next_position()  # Used to calculate if collision == True

        # Update sprite positions
        for sprite, particle_instance in zip(sprites, particles.values()):
            sprite.rect.center = (particle_instance.x, particle_instance.y)

        screen.fill(WHITE)
        sprites.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

if __name__ == "__main__":
    main()


KeyError: 'Na_10'

In [30]:
#Debugging 13-10-2023 == FALSE_V1; LATEST STABLE == V1
#UI == pygame
#Implementado gráfico Max-Boltz
#'future' funciona (inherited 6_10)
#Colisão inelástica e elástica implementada
#Colisão futura implementada em classe Particul
#Reação química feita + grafico conc./tempo
#Fazer: encontrar a lei de velocidade (derivadas)
#Problemas: 
# -não usa grid (menos eficiente)
# -NA_CL_INDEX como global
# -a = 'Na', b = 'Cl'; a = 'Cl', b = 'Na'-> e se forem 3?
# -intersection_position_checker que nao precisa checar tudo ?
# -reações que geram partículas grandes podem gerar intersecção com outra que está indo em direção a esta


import math
import random
import warnings
import pygame
from pygame.locals import *
from sys import exit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Constants
FPS = 60
DT = 1/FPS
ALTURA_CAIXA = 50
LARGURA_CAIXA = 50
NUM_PARTICULAS = 2
GRID_SIZE = 10
FUTURE_FACTOR = 2
TIPO_COLISAO = 'elastic'
COEFICIENTE_RESTITUICAO = 0.0 #coeficiente de restituição

NAME_BOLTZ = 'max-boltz_6_15.png'
NAME_CONCENTRATION_PLOT = 'concentration_6_15.png'
NAME_CSV = 'concentration_data_6_15.csv'

# Indexes de moléculas das reações
NA_CL_INDEX = 0
CHANCE_NACL_GLOBAL = 0.4 #chances de não acontecer a reação

# Colors
WHITE = (255, 255, 255)

class Particula:
    def __init__(self, p_type, massa, raio, vel_x, vel_y, x, y, cor):
        self.p_type = p_type
        self.massa = float(massa)
        self.raio = float(raio)
        self.vel_x = float(vel_x)
        self.vel_y = float(vel_y)
        self.x = float(x)
        self.y = float(y)
        self.x_future = None
        self.y_future = None
        self.cor = cor
        self.sprite_pygame = self.sprite_pygame()

    def deslocar(self):

        self.x += self.vel_x * DT
        self.y += self.vel_y * DT

    def next_position(self):
        self.x_future = self.x + self.vel_x * DT * FUTURE_FACTOR
        self.y_future = self.y + self.vel_y * DT * FUTURE_FACTOR

        if self.x_future < self.raio or self.x_future > LARGURA_CAIXA - self.raio:
            self.vel_x *= -1
        if self.y_future < self.raio or self.y_future > ALTURA_CAIXA - self.raio:
            self.vel_y *= -1

        #print('next_position')

    def sprite_pygame(self):
        circle = pygame.Surface((self.raio * 2, self.raio * 2), pygame.SRCALPHA)
        pygame.draw.circle(circle, self.cor, (self.raio, self.raio), self.raio)
        sprite = pygame.sprite.Sprite()
        sprite.image = circle
        sprite.rect = sprite.image.get_rect(center=(self.x, self.y))
        return sprite

class Grid:
    def __init__(self, width, height, cell_size):
        self.width = width
        self.height = height
        self.cell_size = cell_size
        self.num_cols = int(width / cell_size)
        self.num_rows = int(height / cell_size)
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #cria uma grid nova

    def clear(self):
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #'limpa' a grid

    def index(self, x, y):
        col = int(x / self.cell_size)
        row = int(y / self.cell_size)
        
        # Ensure col and row are within bounds
        col = max(0, min(col, self.num_cols - 1))
        row = max(0, min(row, self.num_rows - 1))
        
        return col + row * self.num_cols

    def insert(self, particle):
        index = self.index(particle.x, particle.y)
        self.grid[index].append(particle)

    def get_particles_in_cell(self, x, y):
        index = self.index(x, y)
        return self.grid[index]

    def update_grid(self, particles):
        self.clear()
        for particle in particles.values():
            self.insert(particle)

    def check_grid_collision(self, particles, grid, sprites):
        particles_to_remove = []  # Create a list to store keys to be removed
        particles_to_add = {}  # Create a dictionary to store items to be added

        particle_dict = particles.copy()  # Create a copy of particles dictionary for iteration

        for name_p1, particle_p1 in particle_dict.items():
            cell_x = int(particle_p1.x / grid.cell_size)
            cell_y = int(particle_p1.y / grid.cell_size)

            for dx, dy in itertools.product([-1, 0, 1], repeat=2):
                neighbor_x = cell_x + dx
                neighbor_y = cell_y + dy
                if 0 <= neighbor_x < grid.num_cols and 0 <= neighbor_y < grid.num_rows:
                    for name_p2, particle_p2 in particle_dict.items():
                        if particle_p1 != particle_p2:
                            if check_collision(particle_p1, particle_p2):
                                new_particles, particles_removed = reaction_collision(
                                    particles, particle_p1, name_p1, particle_p2, name_p2, sprites
                                )
                                # Store keys to be removed
                                particles_to_remove.extend(particles_removed)
                                print('antes_loop',particles_to_remove)

                                # Store items to be added
                                for name_particle_new, new_particle in new_particles.items():
                                    particles_to_add[name_particle_new] = new_particle

        #print('antes_loop_remove', particles_to_remove)
        # Apply the changes after the for loops
        for particle_name_indexed in particles_to_remove:
            print('particulas atuais',particles)
            print('a remover', particle_name_indexed)
            print('remover:', particles_to_remove)
            del particles[particle_name_indexed]

        particle_dict.update(particles_to_add)

        return particle_dict  # Return the updated particles dictionary



def check_collision(p1, p2):

    p1.next_position() #proxima_posicao

    p2.next_position() #proxima_posicao

    distance_future = math.sqrt((p2.x_future - p1.x_future)**2 + (p2.y_future - p1.y_future)**2)
    #distance = math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
    if (distance_future <= p1.raio + p2.raio) == True:
        return True
    else:
        return False


def reaction_collision(particles, p1, p1_name, p2, p2_name, sprites):

    global NA_CL_INDEX

    particles_to_add = {}  # Create a dictionary to store new particles
    particles_to_remove = []  # Create a list to store particles to be removed

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    vel_new_particle = ((p1.massa * v1) + (p2.massa * v2)) / (p1.massa + p2.massa)

    CM_x = (p1.massa * p1.x + p2.massa * p2.x) / (p1.massa + p2.massa)
    CM_y = (p1.massa * p1.y + p2.massa * p2.y) / (p1.massa + p2.massa)

    chance_NaCl_local = random.random()

    if ((p1.p_type == 'Na' and p2.p_type == 'Cl') or (p1.p_type == 'Cl' and p2.p_type == 'Na')) and (chance_NaCl_local > CHANCE_NACL_GLOBAL):
        p_type = 'NaCl'
        massa = (p1.massa + p2.massa)
        raio = 5
        vel_x = vel_new_particle[0]
        vel_y = vel_new_particle[1]
        x = CM_x
        y = CM_y
        cor = (125, 125, 125)
        name_particle = f"{p_type}_{NA_CL_INDEX}"
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)

        sprites.add(particle_instance.sprite_pygame)

        p1.sprite_pygame.kill()
        p2.sprite_pygame.kill()

        #particles_to_add.update({name_particle : particle_instance})
        particles_to_add[name_particle] = particle_instance
        particles_to_remove.extend([p1_name, p2_name])
        NA_CL_INDEX += 1
    else:
        resolve_collision(p1, p2, collision_type=TIPO_COLISAO)
    print('reaction:',particles_to_add,particles_to_remove)
    return particles_to_add, particles_to_remove



def resolve_collision(p1, p2, collision_type='elastic'):
    x1 = np.array([p1.x, p1.y])
    x2 = np.array([p2.x, p2.y])

    #print('arrays_x:', x1, x2)

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    #print('arrays_v:', v1, v2)

    if collision_type == 'elastic' or collision_type == 'elastica':
        #print('elastica')

        C = 1

    if collision_type == 'partial_inelastic' or collision_type == 'parcial_inelastica':
        #print('inelastica')

        C = COEFICIENTE_RESTITUICAO

    new_v1 = v1 - ((((C * p2.massa) + p2.massa) / (p1.massa + p2.massa)) * ((np.dot((v1 - v2),(x1 - x2))) / ((np.linalg.norm(x1 - x2))**2)) * (x1 - x2))
    new_v2 = v2 - ((((C * p1.massa) + p1.massa) / (p1.massa + p2.massa)) * ((np.dot((v2 - v1),(x2 - x1))) / ((np.linalg.norm(x2 - x1))**2)) * (x2 - x1))

    p1.vel_x = new_v1[0]
    p1.vel_y = new_v1[1]
    p2.vel_x = new_v2[0]
    p2.vel_y = new_v2[1]

    #print('resolution',p1.vel_x, p1.vel_y, p2.vel_x, p2.vel_y)

#src_elastica = https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
#src_inelastica = https://physics.stackexchange.com/questions/708495/angle-free-two-dimensional-inelastic-collision-formula




def gerar_particula(n_particulas):
    particulas = {}

    #atomos
    atomos = {
        "Na": {"massa": 22, "raio": 5, "color": (255, 0, 0)}, #approximations
        "Cl": {"massa": 35, "raio": 5, "color": (0, 255, 0)},

    }

    for particle_index in range(1, n_particulas + 1):
        #escolha de distribuição aleatória
        atomos_selecionar = random.choice(list(atomos.keys()))

        attributes = atomos[atomos_selecionar]

        p_type = atomos_selecionar
        massa = attributes["massa"]
        raio = attributes["raio"]
        vel_x = random.uniform(-100, 100) #velocidade inicial não especificada
        vel_y = random.uniform(-100, 100)
        x = random.uniform(raio, LARGURA_CAIXA - raio)
        y = random.uniform(raio, ALTURA_CAIXA - raio)
        #x_future = x + vel_x * DT * FUTURE_FACTOR
        #y_future = y + vel_y * DT * FUTURE_FACTOR
        cor = attributes["color"]
        nome_particula = f"{atomos_selecionar}_{particle_index}" 
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)
        particulas[nome_particula] = particle_instance

    return particulas

def select_particles(particles,sprites):
    particles_update = set()
    particles_remove = []

    #print('entrou pra selected')

    particle_pairs = np.array(list(itertools.combinations(particles.items(), 2)))
    #print('pares',particle_pairs)

    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        #print((name_p1, type(instance_p1)), (name_p2, type(instance_p2)))
        if check_collision(instance_p1, instance_p2):
            #print('foi pra reacted')
            reacted_particles = reaction_collision(particles, instance_p1, name_p1, instance_p2, name_p2,sprites)
            

            if particles != reacted_particles:
                particles_update.update(reacted_particles)
                particles_remove.extend([name_p1, name_p2])

    for particle_name in particles_remove:
        del particles[particle_name]

    particles.update(particles_update)

    return particles



def intersection_pos_checker(particles,iteration_max = 100):
    iteration_count = 0
    intersection = False
    particle_pairs = np.array(list(itertools.permutations(particles.items(), 2)))
    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        if check_collision(instance_p1, instance_p2):
                particle_move = random.choice([[name_p1,instance_p1], [name_p2,instance_p2]])
                new_x = random.uniform(particle_move[1].raio, LARGURA_CAIXA - (particle_move[1].raio))
                new_y = random.uniform(particle_move[1].raio, ALTURA_CAIXA - (particle_move[1].raio))
                particles[particle_move[0]].x = new_x
                particles[particle_move[0]].y = new_y
                iteration_count +=1
                #print(particles[particle_move[0]].y) #verificar coordenadas alteradas

                intersection = True
                if iteration_max == iteration_count:
                    warnings.warn(f'{iteration_max} iterações atingidas. Deve haver partículas sobrepostas. Altere o tamanho da caixa.')
                    return particles
    
    if intersection == True:
        #print('True')
        return intersection_pos_checker(particles)

    if intersection == False:
        #print('False')
        return particles

def create_particle_group_sprites(particles):
    sprites = pygame.sprite.Group()
    for particle_instance in particles.values():
        sprite = particle_instance.sprite_pygame
        sprites.add(sprite)
    #print(sprites)
    return sprites


def chemical_counter(particles,time_passed,df=pd.DataFrame()):

    chem_list = particles.keys()
    #n_moleculas = len(chem_list)

    element_count = {}  # Initialize an empty dictionary to store element counts

    for item in chem_list:
        element = item.split('_')[0]  # Extract the element (part before the underscore)
        if element in element_count:
            element_count[element] += 1  # Increment the count if the element already exists
        else:
            element_count[element] = 1  # Initialize count to 1 if the element is encountered for the first time

   # for key in element_count: #transforma em %
    #    element_count[key] /= n_moleculas


    #df_chem_columns = np.append('Tempo',list(element_count.keys()))
    #print(df_chem_columns)
    chem_array = np.append(time_passed,list(element_count.values()))
    #print(chem_array)
    df_chem = pd.DataFrame(chem_array).T
    df_chem.columns = np.append('Tempo',list(element_count.keys()))

    df_chem = pd.concat([df, df_chem], axis=0, ignore_index=True)

    return df_chem
    

def maxwell_boltzmann_plot(final_speeds,name):
    # Barplot/histograma das velocidades Max-Boltz.
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    plt.hist(final_speeds, bins=25, color='blue', edgecolor='black', alpha=0.7)
    plt.xlabel('Velocidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das velocidades finais')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def concentration_plot(df_chem,name):
    # Lineplot das concentrações/tempo.
    df_chem.to_csv(NAME_CSV, index=False)
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    for y_axis, labels in zip(df_chem.columns[1:],list(df_chem.columns)[1:]):
        plt.plot(df_chem['Tempo'],df_chem[y_axis] ,alpha=0.7, label=labels)
    plt.legend()
    plt.xlabel('Tempo [s]')
    plt.ylabel('Quantidade [un]')
    plt.title('Evolução temporal das concentrações')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def main():
    final_speeds = []
    pygame.init()
    screen = pygame.display.set_mode((LARGURA_CAIXA, ALTURA_CAIXA))
    pygame.display.set_caption("Collision Simulation")

    particles = gerar_particula(NUM_PARTICULAS)
    particles = intersection_pos_checker(particles)
    sprites = create_particle_group_sprites(particles)
    grid = Grid(LARGURA_CAIXA, ALTURA_CAIXA, GRID_SIZE)  # Initialize the grid

    time_passed = 0
    df_chem = pd.DataFrame(columns=['Tempo'])  # Create an empty DataFrame for chemical data

    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                # Get final speeds of all particles
                for particle_instance in particles.values():
                    final_speed = math.sqrt(particle_instance.vel_x ** 2 + particle_instance.vel_y ** 2)
                    final_speeds.append(final_speed)

               # maxwell_boltzmann_plot(final_speeds, name=NAME_BOLTZ)  # Turned off for debugging

                df_chem = df_chem.fillna(0)
                #concentration_plot(df_chem, name=NAME_CONCENTRATION_PLOT)
                exit()

        time_passed += DT

        df_chem = chemical_counter(particles, time_passed, df_chem)
        grid.clear()  # Clear the grid

        for particle_instance in particles.values():
            grid.insert(particle_instance)

        # Use the updated check_grid_collision function
        particles = grid.check_grid_collision(particles, grid, sprites)

        for particle_name, particle_instance in particles.items():
            particle_instance.deslocar()
            particle_instance.next_position()  # Used to calculate if collision == True

        # Update sprite positions
        for sprite, particle_instance in zip(sprites, particles.values()):
            sprite.rect.center = (particle_instance.x, particle_instance.y)

        screen.fill(WHITE)
        sprites.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

if __name__ == "__main__":
    main()


reaction: {} []
antes_loop []
reaction: {'NaCl_0': <__main__.Particula object at 0x0000016543C38490>} ['Cl_1', 'Na_2']
antes_loop ['Cl_1', 'Na_2']
reaction: {'NaCl_1': <__main__.Particula object at 0x0000016546174040>} ['Cl_1', 'Na_2']
antes_loop ['Cl_1', 'Na_2', 'Cl_1', 'Na_2']
reaction: {'NaCl_2': <__main__.Particula object at 0x0000016546308EB0>} ['Cl_1', 'Na_2']
antes_loop ['Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Cl_1', 'Na_2']
reaction: {'NaCl_3': <__main__.Particula object at 0x0000016544831400>} ['Na_2', 'Cl_1']
antes_loop ['Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Na_2', 'Cl_1']
particulas atuais {'Cl_1': <__main__.Particula object at 0x00000165461E5340>, 'Na_2': <__main__.Particula object at 0x00000165460F0460>}
a remover Cl_1
remover: ['Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Na_2', 'Cl_1']
particulas atuais {'Na_2': <__main__.Particula object at 0x00000165460F0460>}
a remover Na_2
remover: ['Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Cl_1', 'Na_2', 'Na_2', 'Cl_1']
particulas at

KeyError: 'Cl_1'

In [3]:
#Debugging 13-10-2023 == FALSE_V1; LATEST STABLE == V1
#UI == pygame
#Implementado gráfico Max-Boltz
#'future' funciona (inherited 6_10)
#Colisão inelástica e elástica implementada
#Colisão futura implementada em classe Particul
#Reação química feita + grafico conc./tempo
#Fazer: encontrar a lei de velocidade (derivadas)
#Problemas: 
# -não usa grid (menos eficiente)
# -NA_CL_INDEX como global
# -a = 'Na', b = 'Cl'; a = 'Cl', b = 'Na'-> e se forem 3?
# -intersection_position_checker que nao precisa checar tudo ?
# -reações que geram partículas grandes podem gerar intersecção com outra que está indo em direção a esta


import math
import random
import warnings
import pygame
from pygame.locals import *
from sys import exit
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools

# Constants
FPS = 60
DT = 1/FPS
ALTURA_CAIXA = 50
LARGURA_CAIXA = 50
NUM_PARTICULAS = 2
GRID_SIZE = 3
FUTURE_FACTOR = 2
TIPO_COLISAO = 'elastic'
COEFICIENTE_RESTITUICAO = 0.0 #coeficiente de restituição

NAME_BOLTZ = 'max-boltz_6_15.png'
NAME_CONCENTRATION_PLOT = 'concentration_6_15.png'
NAME_CSV = 'concentration_data_6_15.csv'

# Indexes de moléculas das reações
NA_CL_INDEX = 0
CHANCE_NACL_GLOBAL = 0.4 #chances de não acontecer a reação

# Colors
WHITE = (255, 255, 255)

class Particula:
    def __init__(self, p_type, massa, raio, vel_x, vel_y, x, y, cor):
        self.p_type = p_type
        self.massa = float(massa)
        self.raio = float(raio)
        self.vel_x = float(vel_x)
        self.vel_y = float(vel_y)
        self.x = float(x)
        self.y = float(y)
        self.x_future = None
        self.y_future = None
        self.cor = cor
        self.sprite_pygame = self.sprite_pygame()

    def deslocar(self):

        self.x += self.vel_x * DT
        self.y += self.vel_y * DT

    def next_position(self):
        self.x_future = self.x + self.vel_x * DT * FUTURE_FACTOR
        self.y_future = self.y + self.vel_y * DT * FUTURE_FACTOR

        if self.x_future < self.raio or self.x_future > LARGURA_CAIXA - self.raio:
            self.vel_x *= -1
        if self.y_future < self.raio or self.y_future > ALTURA_CAIXA - self.raio:
            self.vel_y *= -1

        #print('next_position')

    def sprite_pygame(self):
        circle = pygame.Surface((self.raio * 2, self.raio * 2), pygame.SRCALPHA)
        pygame.draw.circle(circle, self.cor, (self.raio, self.raio), self.raio)
        sprite = pygame.sprite.Sprite()
        sprite.image = circle
        sprite.rect = sprite.image.get_rect(center=(self.x, self.y))
        return sprite

class Grid:
    def __init__(self, width, height, cell_size):
        self.width = width
        self.height = height
        self.cell_size = cell_size
        self.num_cols = int(width / cell_size)
        self.num_rows = int(height / cell_size)
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #cria uma grid nova

    def clear(self):
        self.grid = [[] for _ in range(self.num_cols * self.num_rows)] #'limpa' a grid

    def index(self, x, y):
        col = int(x / self.cell_size)
        row = int(y / self.cell_size)
        
        # Ensure col and row are within bounds
        col = max(0, min(col, self.num_cols - 1))
        row = max(0, min(row, self.num_rows - 1))
        
        return col + row * self.num_cols

    def insert(self, particle):
        index = self.index(particle.x, particle.y)
        print(particle.x, particle.y)
        self.grid[index].append(particle)

    def get_particles_in_cell(self, x, y):
        index = self.index(x, y)
        return self.grid[index]

    def update_grid(self, particles):
        self.clear()
        for particle in particles.values():
            self.insert(particle)

    def check_grid_collision(self, particles, grid, sprites):
        particles_to_remove = set()  # Create a set to store keys to be removed
        particles_to_add = {}  # Create a dictionary to store items to be added

        particle_dict = particles.copy()  # Create a copy of particles dictionary for iteration

        for name_p1, particle_p1 in particle_dict.items():
            cell_x = int(particle_p1.x / grid.cell_size)
            cell_y = int(particle_p1.y / grid.cell_size)

            for dx, dy in itertools.product([-1, 0, 1], repeat=2):
                neighbor_x = cell_x + dx
                neighbor_y = cell_y + dy
                if 0 <= neighbor_x < grid.num_cols and 0 <= neighbor_y < grid.num_rows:
                    for name_p2, particle_p2 in particle_dict.items():
                        if particle_p1 != particle_p2:
                            if check_collision(particle_p1, particle_p2):
                                new_particles, particles_removed = reaction_collision(
                                particles, particle_p1, name_p1, particle_p2, name_p2, sprites
                            )
                            # Store keys to be removed in the set
                                particles_to_remove.update(particles_removed)

                            # Store items to be added
                                for name_particle_new, new_particle in new_particles.items():
                                    particles_to_add[name_particle_new] = new_particle

        # Apply the changes after the for loops
        for particle_name_indexed in particles_to_remove:
            del particles[particle_name_indexed]

        particle_dict.update(particles_to_add)

        return particle_dict  # Return the updated particles dictionary


def check_collision(p1, p2):

    p1.next_position() #proxima_posicao

    p2.next_position() #proxima_posicao

    distance_future = math.sqrt((p2.x_future - p1.x_future)**2 + (p2.y_future - p1.y_future)**2)
    #distance = math.sqrt((p2.x - p1.x)**2 + (p2.y - p1.y)**2)
    if (distance_future <= p1.raio + p2.raio) == True:
        return True
    else:
        return False


def reaction_collision(particles, p1, p1_name, p2, p2_name, sprites):

    global NA_CL_INDEX

    particles_to_add = {}  # Create a dictionary to store new particles
    particles_to_remove = []  # Create a list to store particles to be removed

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    vel_new_particle = ((p1.massa * v1) + (p2.massa * v2)) / (p1.massa + p2.massa)

    CM_x = (p1.massa * p1.x + p2.massa * p2.x) / (p1.massa + p2.massa)
    CM_y = (p1.massa * p1.y + p2.massa * p2.y) / (p1.massa + p2.massa)

    chance_NaCl_local = random.random()

    if ((p1.p_type == 'Na' and p2.p_type == 'Cl') or (p1.p_type == 'Cl' and p2.p_type == 'Na')) and (chance_NaCl_local > CHANCE_NACL_GLOBAL):
        p_type = 'NaCl'
        massa = (p1.massa + p2.massa)
        raio = 5
        vel_x = vel_new_particle[0]
        vel_y = vel_new_particle[1]
        x = CM_x
        y = CM_y
        cor = (125, 125, 125)
        name_particle = f"{p_type}_{NA_CL_INDEX}"
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)

        sprites.add(particle_instance.sprite_pygame)

        p1.sprite_pygame.kill()
        p2.sprite_pygame.kill()

        #particles_to_add.update({name_particle : particle_instance})
        particles_to_add[name_particle] = particle_instance
        particles_to_remove.extend([p1_name, p2_name])
        NA_CL_INDEX += 1
    else:
        resolve_collision(p1, p2, collision_type=TIPO_COLISAO)
    #print('reaction:',particles_to_add,particles_to_remove)
    return particles_to_add, particles_to_remove



def resolve_collision(p1, p2, collision_type='elastic'):
    x1 = np.array([p1.x, p1.y])
    x2 = np.array([p2.x, p2.y])

    #print('arrays_x:', x1, x2)

    v1 = np.array([p1.vel_x, p1.vel_y])
    v2 = np.array([p2.vel_x, p2.vel_y])

    #print('arrays_v:', v1, v2)

    if collision_type == 'elastic' or collision_type == 'elastica':
        #print('elastica')

        C = 1

    if collision_type == 'partial_inelastic' or collision_type == 'parcial_inelastica':
        #print('inelastica')

        C = COEFICIENTE_RESTITUICAO

    new_v1 = v1 - ((((C * p2.massa) + p2.massa) / (p1.massa + p2.massa)) * ((np.dot((v1 - v2),(x1 - x2))) / ((np.linalg.norm(x1 - x2))**2)) * (x1 - x2))
    new_v2 = v2 - ((((C * p1.massa) + p1.massa) / (p1.massa + p2.massa)) * ((np.dot((v2 - v1),(x2 - x1))) / ((np.linalg.norm(x2 - x1))**2)) * (x2 - x1))

    p1.vel_x = new_v1[0]
    p1.vel_y = new_v1[1]
    p2.vel_x = new_v2[0]
    p2.vel_y = new_v2[1]

    #print('resolution',p1.vel_x, p1.vel_y, p2.vel_x, p2.vel_y)

#src_elastica = https://en.wikipedia.org/wiki/Elastic_collision#Two-dimensional_collision_with_two_moving_objects
#src_inelastica = https://physics.stackexchange.com/questions/708495/angle-free-two-dimensional-inelastic-collision-formula




def gerar_particula(n_particulas):
    particulas = {}

    #atomos
    atomos = {
        "Na": {"massa": 22, "raio": 5, "color": (255, 0, 0)}, #approximations
        "Cl": {"massa": 35, "raio": 5, "color": (0, 255, 0)},

    }

    for particle_index in range(1, n_particulas + 1):
        #escolha de distribuição aleatória
        atomos_selecionar = random.choice(list(atomos.keys()))

        attributes = atomos[atomos_selecionar]

        p_type = atomos_selecionar
        massa = attributes["massa"]
        raio = attributes["raio"]
        vel_x = random.uniform(-100, 100) #velocidade inicial não especificada
        vel_y = random.uniform(-100, 100)
        x = random.uniform(raio, LARGURA_CAIXA - raio)
        y = random.uniform(raio, ALTURA_CAIXA - raio)
        #x_future = x + vel_x * DT * FUTURE_FACTOR
        #y_future = y + vel_y * DT * FUTURE_FACTOR
        cor = attributes["color"]
        nome_particula = f"{atomos_selecionar}_{particle_index}" 
        particle_instance = Particula(p_type, massa, raio, vel_x, vel_y, x, y, cor)
        particulas[nome_particula] = particle_instance

    return particulas

def select_particles(particles,sprites):
    particles_update = set()
    particles_remove = []

    #print('entrou pra selected')

    particle_pairs = np.array(list(itertools.combinations(particles.items(), 2)))
    #print('pares',particle_pairs)

    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        #print((name_p1, type(instance_p1)), (name_p2, type(instance_p2)))
        if check_collision(instance_p1, instance_p2):
            #print('foi pra reacted')
            reacted_particles = reaction_collision(particles, instance_p1, name_p1, instance_p2, name_p2,sprites)
            

            if particles != reacted_particles:
                particles_update.update(reacted_particles)
                particles_remove.extend([name_p1, name_p2])

    for particle_name in particles_remove:
        del particles[particle_name]

    particles.update(particles_update)

    return particles



def intersection_pos_checker(particles,iteration_max = 100):
    iteration_count = 0
    intersection = False
    particle_pairs = np.array(list(itertools.permutations(particles.items(), 2)))
    for (name_p1, instance_p1), (name_p2, instance_p2) in particle_pairs:
        if check_collision(instance_p1, instance_p2):
                particle_move = random.choice([[name_p1,instance_p1], [name_p2,instance_p2]])
                new_x = random.uniform(particle_move[1].raio, LARGURA_CAIXA - (particle_move[1].raio))
                new_y = random.uniform(particle_move[1].raio, ALTURA_CAIXA - (particle_move[1].raio))
                particles[particle_move[0]].x = new_x
                particles[particle_move[0]].y = new_y
                iteration_count +=1
                #print(particles[particle_move[0]].y) #verificar coordenadas alteradas

                intersection = True
                if iteration_max == iteration_count:
                    warnings.warn(f'{iteration_max} iterações atingidas. Deve haver partículas sobrepostas. Altere o tamanho da caixa.')
                    return particles
    
    if intersection == True:
        #print('True')
        return intersection_pos_checker(particles)

    if intersection == False:
        #print('False')
        return particles

def create_particle_group_sprites(particles):
    sprites = pygame.sprite.Group()
    for particle_instance in particles.values():
        sprite = particle_instance.sprite_pygame
        sprites.add(sprite)
    #print(sprites)
    return sprites


def chemical_counter(particles,time_passed,df=pd.DataFrame()):

    chem_list = particles.keys()
    #n_moleculas = len(chem_list)

    element_count = {}  # Initialize an empty dictionary to store element counts

    for item in chem_list:
        element = item.split('_')[0]  # Extract the element (part before the underscore)
        if element in element_count:
            element_count[element] += 1  # Increment the count if the element already exists
        else:
            element_count[element] = 1  # Initialize count to 1 if the element is encountered for the first time

   # for key in element_count: #transforma em %
    #    element_count[key] /= n_moleculas


    #df_chem_columns = np.append('Tempo',list(element_count.keys()))
    #print(df_chem_columns)
    chem_array = np.append(time_passed,list(element_count.values()))
    #print(chem_array)
    df_chem = pd.DataFrame(chem_array).T
    df_chem.columns = np.append('Tempo',list(element_count.keys()))

    df_chem = pd.concat([df, df_chem], axis=0, ignore_index=True)

    return df_chem
    

def maxwell_boltzmann_plot(final_speeds,name):
    # Barplot/histograma das velocidades Max-Boltz.
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    plt.hist(final_speeds, bins=25, color='blue', edgecolor='black', alpha=0.7)
    plt.xlabel('Velocidade')
    plt.ylabel('Frequência')
    plt.title('Histograma das velocidades finais')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def concentration_plot(df_chem,name):
    # Lineplot das concentrações/tempo.
    df_chem.to_csv(NAME_CSV, index=False)
    fig, ax = plt.subplots(figsize=(6, 6))  # Cria caixa para plot
    for y_axis, labels in zip(df_chem.columns[1:],list(df_chem.columns)[1:]):
        plt.plot(df_chem['Tempo'],df_chem[y_axis] ,alpha=0.7, label=labels)
    plt.legend()
    plt.xlabel('Tempo [s]')
    plt.ylabel('Quantidade [un]')
    plt.title('Evolução temporal das concentrações')
    plt.grid(True)
    fig.tight_layout()
    fig.savefig(name)  # Salva a figura
    plt.show()


def main():
    final_speeds = []
    pygame.init()
    screen = pygame.display.set_mode((LARGURA_CAIXA, ALTURA_CAIXA))
    pygame.display.set_caption("Collision Simulation")

    particles = gerar_particula(NUM_PARTICULAS)
    particles = intersection_pos_checker(particles)
    sprites = create_particle_group_sprites(particles)
    grid = Grid(LARGURA_CAIXA, ALTURA_CAIXA, GRID_SIZE)  # Initialize the grid

    time_passed = 0
    df_chem = pd.DataFrame(columns=['Tempo'])  # Create an empty DataFrame for chemical data

    clock = pygame.time.Clock()

    while True:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                # Get final speeds of all particles
                for particle_instance in particles.values():
                    final_speed = math.sqrt(particle_instance.vel_x ** 2 + particle_instance.vel_y ** 2)
                    final_speeds.append(final_speed)

               # maxwell_boltzmann_plot(final_speeds, name=NAME_BOLTZ)  # Turned off for debugging

                df_chem = df_chem.fillna(0)
                #concentration_plot(df_chem, name=NAME_CONCENTRATION_PLOT)
                exit()

        time_passed += DT

        df_chem = chemical_counter(particles, time_passed, df_chem)
        grid.clear()  # Clear the grid

        for particle_instance in particles.values():
            grid.insert(particle_instance)

        # Use the updated check_grid_collision function
        particles = grid.check_grid_collision(particles, grid, sprites)

        for particle_name, particle_instance in particles.items():
            particle_instance.deslocar()
            particle_instance.next_position()  # Used to calculate if collision == True

        # Update sprite positions
        for sprite, particle_instance in zip(sprites, particles.values()):
            sprite.rect.center = (particle_instance.x, particle_instance.y)

        screen.fill(WHITE)
        sprites.draw(screen)

        pygame.display.flip()
        clock.tick(FPS)

if __name__ == "__main__":
    main()


7.848562272553141 24.17309991117452
24.198131969634392 35.11707219958582
8.03780475979127 25.562132963561883
24.13537244266969 36.06017260318766
8.227047247029398 26.951166015949244
24.07261291570499 37.0032730067895
8.416289734267528 28.340199068336606
24.00985338874029 37.94637341039134
8.605532221505657 29.729232120723967
23.94709386177559 38.88947381399318
8.794774708743786 31.11826517311133
23.884334334810887 39.83257421759502
8.984017195981915 32.507298225498694
23.821574807846186 40.775674621196856
9.173259683220044 33.896331277886055
23.758815280881485 41.718775024798695
9.362502170458173 35.28536433027342
23.696055753916784 42.661875428400535
9.551744657696302 36.67439738266078
23.633296226952083 43.604975832002374
9.740987144934431 38.06343043504814
23.570536699987382 42.661875428400535
9.93022963217256 39.4524634874355
23.50777717302268 41.718775024798695
10.11947211941069 40.84149653982286
23.44501764605798 40.775674621196856
10.308714606648818 42.230529592210225
23.3822581

C:\Users\gustavo220061\AppData\Local\Temp\ipykernel_13620\853145628.py:240: RuntimeWarning: invalid value encountered in scalar divide
  new_v1 = v1 - ((((C * p2.massa) + p2.massa) / (p1.massa + p2.massa)) * ((np.dot((v1 - v2),(x1 - x2))) / ((np.linalg.norm(x1 - x2))**2)) * (x1 - x2))
C:\Users\gustavo220061\AppData\Local\Temp\ipykernel_13620\853145628.py:241: RuntimeWarning: invalid value encountered in scalar divide
  new_v2 = v2 - ((((C * p1.massa) + p1.massa) / (p1.massa + p2.massa)) * ((np.dot((v2 - v1),(x2 - x1))) / ((np.linalg.norm(x2 - x1))**2)) * (x2 - x1))


ValueError: cannot convert float NaN to integer

In [1]:
#ideia de grid
#o último código é o mais recente
#problema: o código anterior cria tanta partícula que no fim a posição de uma nova fica muito próximo de zero ~ Nan
#isso ocorre pois a grid faz uns loop/iterções que não precisa
#por isso que eu tive que usar set()
#como resolver ?
class Grid:
    def __init__(self, width, height, cell_size):
        self.width = width
        self.height = height
        self.cell_size = cell_size
        self.grid = [[[] for _ in range(width // cell_size)] for _ in range(height // cell_size)]
        self.particle_to_cell = {}  # Keep track of which cell each particle belongs to

    def place_particle(self, particle):
        if particle in self.particle_to_cell:
            # Remove particle from its previous cell
            prev_cell_x, prev_cell_y = self.particle_to_cell[particle]
            self.grid[prev_cell_y][prev_cell_x].remove(particle)

        x, y = particle.x, particle.y
        cell_x = int(x / self.cell_size)
        cell_y = int(y / self.cell_size)
        self.grid[cell_y][cell_x].append(particle)
        self.particle_to_cell[particle] = (cell_x, cell_y)

    def check_grid_collision(self, particles, sprites):
        particles_to_remove = set()
        particles_to_add = {}

        for particle_name, particle in particles.items():
            cell_x, cell_y = self.particle_to_cell[particle]

            for dx, dy in itertools.product([-1, 0, 1], repeat=2):
                neighbor_x = cell_x + dx
                neighbor_y = cell_y + dy

                if 0 <= neighbor_x < len(self.grid[0]) and 0 <= neighbor_y < len(self.grid):
                    for neighbor_particle in self.grid[neighbor_y][neighbor_x]:
                        if neighbor_particle != particle:
                            if check_collision(particle, neighbor_particle):
                                new_particles, particles_removed = reaction_collision(
                                    particles, particle, particle_name, neighbor_particle, neighbor_particle_name, sprites
                                )
                                particles_to_remove.update(particles_removed)
                                for name_particle_new, new_particle in new_particles.items():
                                    particles_to_add[name_particle_new] = new_particle

        for particle_name_indexed in particles_to_remove:
            del particles[particle_name_indexed]

        particles.update(particles_to_add)

    def clear_grid(self):
        for row in self.grid:
            for cell in row:
                cell.clear()
        self.particle_to_cell.clear()

pygame 2.5.1 (SDL 2.28.2, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\venv\ilumpy-VS\lib\site-packages\IPython\core\interactiveshell.py:3516: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
